### Task [99](https://github.com/salgo60/ProjectOutdoorGyms/issues/99) "uppdater Stockholms utegym"
Verkar som det händer lite dags att uppdater...  se även [#114](https://github.com/salgo60/ProjectOutdoorGyms/issues/114) video med feedback [länk video](https://www.youtube.com/watch?v=8FGFc5rljhc)

* API [utegym Stockholm](https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=0eb151d8-2bf9-4a3d-b644-5fb51398779c)
  * [API call](https://apigw.stockholm.se/NoAuth/VirtualhittaserviceDMZ/Rest/serviceunits?&filter%5Bservicetype.id%5D=122&page%5Blimit%5D=1500&page%5Boffset%5D=0&sort=name)
  
* [denna notebook](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/Stockholm%20outdoor%20gym.ipynb) 
  * [GITHUB yta](https://github.com/salgo60/ProjectOutdoorGyms) / [Wikidata projektyta utegym]()

* se även task där vi vill ha id hos Naturkartan i detta API [ProjectOutdoorGyms/issues/100](https://github.com/salgo60/ProjectOutdoorGyms/issues/100)
  * alla issues med utegym och [Stockholms stad länk](https://github.com/salgo60/ProjectOutdoorGyms/issues?q=label%3A%22Stockholms+stad%22+)
  * [senaste 10 Stockholms stad](https://w.wiki/5Ykn) och [planerade utegym Sverige](https://w.wiki/3w$Y) vi hittat

**[Video hur denna Notebook fungerar](https://youtu.be/8FGFc5rljhc)**
* Stockholms stad har koordinat SWEREF99_18_00 kan konverteras på [rl.se/utm](https://rl.se/utm)

<div>
<img src="https://blogger.googleusercontent.com/img/a/AVvXsEhVnfL1xB7a-E5KBkV6N4QD1X3KpTXlGYEmEiaITQF-a_Pd3OTJAYDHEhAfExIekPHQtA-WQngCfofp8Ry-HbKimQwiHNW_DIBiVgbTiesqHA3GgU1p7xpN_dIjzam7FX_Ehwol0P1DIcEXTn2LH0A-qLtACzr1LUKxu0Uj_XQymI6tcNoLvEhzxVtp5A" width="500"/>
</div>

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-11-07 19:49:44.190882


In [2]:
url="https://apigw.stockholm.se/NoAuth/VirtualhittaserviceDMZ/Rest/serviceunits?&filter[servicetype.id]=122&page[limit]=1500&page[offset]=0&sort=name"
import urllib3, json
import pandas as pd
from tqdm import tqdm
http = urllib3.PoolManager()
try:
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
except:
    print (r.status, url)


In [3]:
SthmlStadData = pd.json_normalize(data["data"])
SthmlStadData

,type,id,attributes.name,attributes.location.north,attributes.location.east,attributes.address,attributes.isPublished,attributes.created,attributes.changed,relationships.image.data.type,relationships.image.data.id,relationships.image.links.self,relationships.image.links.related,relationships.serviceType.data.type,relationships.serviceType.data.id,relationships.serviceType.links.self,relationships.serviceType.links.related,relationships.regions.data,links.self
0,serviceunits,12756,Akalla Gårds utegym,6588557,145156,Imatragatan 230,True,2019-08-22,2022-02-23,files,6c9ed8be-3f0d-4c93-af16-ba44de88b9ae,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '8'}]",https://apigw.stockholm.se/noauth/virtualhitta...
1,serviceunits,15335,Alviks strands utegym,6579602,149177,Gustavslundsvägen,True,2022-03-11,2022-03-11,files,88ba9596-00d0-41f3-b10d-a12f384d3015,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '1'}]",https://apigw.stockholm.se/noauth/virtualhitta...
2,serviceunits,14997,Aromen utegym,6576875,151468,Ingenjörsvägen 25,True,2021-01-21,2021-09-24,files,696c2e97-3f81-4efa-86c8-40d7eb8e0010,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '16'}]",https://apigw.stockholm.se/noauth/virtualhitta...
3,serviceunits,15474,Bandängens utegym,6572122,152562,Harpsundsvägen 52,True,2022-08-03,2022-08-15,files,4c916768-f603-4861-b28a-f9866fcb2a78,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '2'}]",https://apigw.stockholm.se/noauth/virtualhitta...
4,serviceunits,15453,Beckomberga utegym,6582918,144581,Grafitgränd,True,2022-06-30,2022-06-30,files,f7bb6398-bda1-4ee7-ac58-9ae463dbed53,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '1'}]",https://apigw.stockholm.se/noauth/virtualhitta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,serviceunits,13675,Ärvinge utegym,6587132,146225,NaN,True,2019-10-22,2022-01-18,files,2ac1f6e0-4102-4cb7-ad19-d9a74aaaf91d,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '8'}]",https://apigw.stockholm.se/noauth/virtualhitta...
93,serviceunits,12762,Ågestas utegym,6567465,155986,NaN,True,2019-08-22,2019-10-28,files,c7a2a9b5-adfb-410b-9ce8-de4f864a62b2,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '15'}]",https://apigw.stockholm.se/noauth/virtualhitta...
94,serviceunits,13662,Årstaskogens utegym,6576427,153374,NaN,True,2019-10-22,2021-02-05,files,54f29ec3-a624-46ac-9b31-2c5a6f6c96ab,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '2'}]",

In [4]:
SthmlStadData.to_csv("Stockholmutegym.csv")

In [16]:
SthmlStadData.sort_values(by=['attributes.changed'], inplace=True) 

In [22]:
SthmlStadData.tail(10)

,type,id,attributes.name,attributes.location.north,attributes.location.east,attributes.address,attributes.isPublished,attributes.created,attributes.changed,relationships.image.data.type,relationships.image.data.id,relationships.image.links.self,relationships.image.links.related,relationships.serviceType.data.type,relationships.serviceType.data.id,relationships.serviceType.links.self,relationships.serviceType.links.related,relationships.regions.data,links.self
46,serviceunits,13637,Liljeholmens utegym,6577274,151181,"Liljeholmsgränd 15, Stockholm",True,2019-10-21,2022-10-20,files,66416431-bea6-4624-8355-88807b8d0e05,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '16'}]",https://apigw.stockholm.se/noauth/virtualhitta...
63,serviceunits,13645,Rålambshovsparkens utegym,6579345,151442,Rålambshovsparken,True,2019-10-22,2022-10-26,files,7d0096f1-68fe-470a-a7b3-8d3c530fbed2,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '6'}]",https://apigw.stockholm.se/noauth/virtualhitta...
83,serviceunits,13658,Vanadislundens utegym,6581388,153013,Vanadislunden i korsningen Frejgatan/Sveavägen,True,2019-10-22,2022-10-26,files,803da561-e48f-4dbc-bed7-ee29451e12a4,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '7'}]",https://apigw.stockholm.se/noauth/virtualhitta...
68,serviceunits,13648,Smedsuddsbadets utegym,6578963,151203,Smedsuddsbadet,True,2019-10-22,2022-10-27,files,b80b5fcf-0810-4d18-be85-e0ed5d328b26,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '6'}]",https://apigw.stockholm.se/noauth/virtualhitta...
32,serviceunits,14749,Humlans utegym,6580583,154009,Humlegården,True,2020-06-03,2022-10-27,files,64ef5d6e-ddb0-48e5-b8ef-37317f6f7790,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '14'}]",https://apigw.stockholm.se/noauth/virtualhitta...
84,serviceunits,13659,Vasaparkens utegym,6580442,152307,"Vasaparken, mot Torsgatan",True,2019-10-22,2022-10-27,files,da15fdae-81e7-411b-ac79-a6661392aef9,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '7'}]",https://apigw.stockholm.se/noauth/virtualhitta...
41,serviceunits,13634,Kronobergsparkens utegym,6579690,152064,Kronobergsparken,True,2019-10-21,2022-10-27,files,42b402d8-c9a8-4e8f-93bd-145077a03471,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '6'}]",https://apigw.stockholm.se/noauth/virtualhitta...
8,serviceunits,15486,Broparkens utegym,6578472,149898,Stora Essingen,True,2022-09-22,2022-10-27,files,d7178470-ed29-4ed9-b621-c439733c93b6,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta

### SPARQL
Hämta utegym i WD för Stockholm
* [SPARQL](https://w.wiki/59Mm) Wikidata 

In [5]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

querySthlm = """#title:Outdoor gyms Stockholm in WD
SELECT distinct ?inception ?item ?itemLabel ?www   ?servicenr ?API WHERE {
  ?item wdt:P6104 wd:Q107186275.
  OPTIONAL {?item wdt:P571 ?inception}

  OPTIONAL {?item wdt:P217 ?servicenr}
  BIND(URI(CONCAT("https://apigw.stockholm.se/noauth/virtualhittaservicedmz/rest/serviceunits/",?servicenr)) AS ?API)
                 
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en,sv,de" . 
  }
  
?item wdt:P571 ?date. # date
OPTIONAL { ?item wdt:P18 ?img. }
?item wdt:P131 wd:Q506250.
OPTIONAL { ?item wdt:P856 ?www. }
OPTIONAL { ?item wdt:P625 ?coord. }
OPTIONAL { ?item wdt:P217 ?id. }
} order by desc(?date)"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDtotSthlmOutdoorGym = get_sparql_dataframe(endpoint_url, querySthlm)

WDtotSthlmOutdoorGym


,inception,item,itemLabel,www,servicenr,API
0,2023-01-01T00:00:00Z,http://www.wikidata.org/entity/Q108257729,"Kista äng, outdoor gym",https://vaxer.stockholm/projekt/varbergstoppen...,None,None
1,2022-11-01T00:00:00Z,http://www.wikidata.org/entity/Q108183619,"Väständaparken, outdoor gym",https://vaxer.stockholm/nyheter/2020/06/byggst...,None,None
2,2022-09-22T00:00:00Z,http://www.wikidata.org/entity/Q113481571,Broparken outdoor gym,https://motionera.stockholm/hitta-utegym/utegy...,15486,https://apigw.stockholm.se/noauth/virtualhitta...
3,2022-08-03T00:00:00Z,http://www.wikidata.org/entity/Q113468344,Bandängens outdoor gym,https://motionera.stockholm/hitta-utegym/utegy...,15474,https://apigw.stockholm.se/noauth/virtualhitta...
4,2022-08-01T00:00:00Z,http://www.wikidata.org/entity/Q113512532,Nockebyhus outdoorgym,None,None,None
...,...,...,...,...,...,...
105,2019-08-22T00:00:00Z,http://www.wikidata.org/entity/Q107208370,"Ågestas, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12762,https://apigw.stockholm.se/noauth/virtualhitta...
106,2019-08-22T00:00:00Z,http://www.wikidata.org/entity/Q107208373,"Östbergas, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12763,https://apigw.stockholm.se/noauth/virtualhitta...
107,2019-08-21T00:00:00Z,http://www.wikidata.org/entity/Q107208313,"Eriksdals, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12752,https://apigw.stockholm.se/noauth/virtualhitta...
108,2019-08-21T00:00:00Z,http://www.wikidata.org/entity/Q107208315,"Farsta, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12753,https://apigw.stockholm.se/noauth/virtualhitta...


* Looks like we have some planned in WD without "servicenr" --> maybe they need more care  

* Would be great if a planned outdoor gym got a servicenr and also a webpage about an ipject had this servicenr

In [6]:
WDtotSthlmOutdoorGym.isnull().sum()

inception     0
item          0
itemLabel     0
www           1
servicenr    10
API          10
dtype: int64

In [7]:
WDtotSthlmOutdoorGymMissService = WDtotSthlmOutdoorGym[WDtotSthlmOutdoorGym["servicenr"].isna()]

In [8]:
WDtotSthlmOutdoorGymMissService

,inception,item,itemLabel,www,servicenr,API
0,2023-01-01T00:00:00Z,http://www.wikidata.org/entity/Q108257729,"Kista äng, outdoor gym",https://vaxer.stockholm/projekt/varbergstoppen...,None,None
1,2022-11-01T00:00:00Z,http://www.wikidata.org/entity/Q108183619,"Väständaparken, outdoor gym",https://vaxer.stockholm/nyheter/2020/06/byggst...,None,None
4,2022-08-01T00:00:00Z,http://www.wikidata.org/entity/Q113512532,Nockebyhus outdoorgym,None,None,None
5,2022-08-01T00:00:00Z,http://www.wikidata.org/entity/Q108180708,Teaterparken outdoor gym,https://vaxer.stockholm/globalassets/omraden/h...,None,None
7,2022-05-01T00:00:00Z,http://www.wikidata.org/entity/Q108209053,"Stora Essinge, outdoor gym",https://vaxer.stockholm/projekt/stora-essingen...,None,None
8,2022-05-01T00:00:00Z,http://www.wikidata.org/entity/Q108209053,"Stora Essinge, outdoor gym",https://vaxer.stockholm/globalassets/projekt/k...,None,None
21,2021-09-01T00:00:00Z,http://www.wikidata.org/entity/Q108307465,"Kräftriket, outdoor gym",https://www.su.se/medarbetare/r%C3%A5d-st%C3%B...,None,None
93,2019-10-21T00:00:00Z,http://www.wikidata.org/entity/Q108159294,"Lappis, outdoor gym",https://sssb.se/bo-hos-oss/utegym/,None,None
94,2019-10-21T00:00:00Z,http://www.wikidata.org/entity/Q108159615,"Strix, outdoor gym",https://sssb.se/bo-hos-oss/utegym/,None,None
99,2019-08-22T00:00:00Z,http://www.wikidata.org/entity/Q107208359,"Sätra idrottsplats, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,None,None


In [9]:
#Compare WDtotSthlmOutdoorGym["servicenr"] and SthmlStadData[id]
WDtotSthlmOutdoorGym["servicenr"]

0       None
1       None
2      15486
3      15474
4       None
       ...  
105    12762
106    12763
107    12752
108    12753
109    12754
Name: servicenr, Length: 110, dtype: object

In [10]:
SthmlStadDataNew = SthmlStadData.rename(columns={'id': 'servicenr'})
#SthmlStadData["servicenr"]
SthmlStadDataNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   type                                     97 non-null     object
 1   servicenr                                97 non-null     object
 2   attributes.name                          97 non-null     object
 3   attributes.location.north                97 non-null     int64 
 4   attributes.location.east                 97 non-null     int64 
 5   attributes.address                       58 non-null     object
 6   attributes.isPublished                   97 non-null     bool  
 7   attributes.created                       97 non-null     object
 8   attributes.changed                       96 non-null     object
 9   relationships.image.data.type            97 non-null     object
 10  relationships.image.data.id              97 non-null     object


In [11]:
OutdoorGym_diff = pd.merge(SthmlStadDataNew, WDtotSthlmOutdoorGym, how='outer', indicator='Exist')

In [12]:
OutdoorGym_diff["Exist"].value_counts()

both          99
right_only    11
left_only      0
Name: Exist, dtype: int64

In [13]:
MissingGyms = OutdoorGym_diff[OutdoorGym_diff["Exist"] == "left_only"]

In [14]:
MissingGyms

,type,servicenr,attributes.name,attributes.location.north,attributes.location.east,attributes.address,attributes.isPublished,attributes.created,attributes.changed,relationships.image.data.type,...,relationships.serviceType.links.self,relationships.serviceType.links.related,relationships.regions.data,links.self,inception,item,itemLabel,www,API,Exist


In [15]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-11-07 19:49:47.999697
Time elapsed (hh:mm:ss.ms) 0:00:03.809485
